# Membershif Inference attack

To run the notebook:
```jupyter nbconvert --execute  -to notebook --inplace Membership_inference.ipynb```

## Dataset Selection
select a dataset

In [3]:
# from ipywidgets import interact, widgets
import ipywidgets as widgets

dataset = 'purchase-100'
dataset_list = ['purchase-10', 'purchase-20', 'purchase-50', 'purchase-100', 'texas', 'MNIST', 'synthetic', 'Census', 'DNA']

from data_preprocessing import *

## Attack Parameters
set the attack parameters for the test below

## Defence Parameters
set the defence parameters below

most of the settings can be set in config.py!

In [9]:
from federated_xgboost.FLTreeHMulti import H_PlainFedXGBoost # USE HMULTI
from federated_xgboost.XGBoostCommon import XgboostLearningParam, PARTY_ID 
from config import CONFIG, dataset, rank, logger, comm
from data_structure.DataBaseStructure import QuantileParam
from algo.LossFunction import LeastSquareLoss, LogLoss, SoftMax


XgboostLearningParam.LOSS_FUNC = SoftMax()
XgboostLearningParam.LOSS_TERMINATE = 50 # this is not done right now in the multi-class approach!
XgboostLearningParam.GAMMA = CONFIG["gamma"]
XgboostLearningParam.LAMBDA = CONFIG["lambda"]
QuantileParam.epsilon = QuantileParam.epsilon
QuantileParam.thres_balance = 0.3

XgboostLearningParam.N_TREES = CONFIG["MAX_TREE"]
XgboostLearningParam.MAX_DEPTH = CONFIG["MAX_DEPTH"]


if CONFIG["model"] == "PlainXGBoost":
    model = H_PlainFedXGBoost(XgboostLearningParam.N_TREES, 10)

logger.warning("TestInfo, {0}".format(CONFIG))
logger.warning("XGBoostParameter, nTree: %d, maxDepth: %d, lambda: %f, gamma: %f", 
XgboostLearningParam.N_TREES, XgboostLearningParam.MAX_DEPTH, XgboostLearningParam.LAMBDA, XgboostLearningParam.GAMMA)
logger.warning("QuantileParameter, eps: %f, thres: %f", QuantileParam.epsilon, QuantileParam.thres_balance)


## Assesment Parameters
set plotting and other evaluating settings below

## Membership Inference Attack
How the membership Inference attack is done can be seen below

In [ ]:
if rank != -1:
    if CONFIG["dataset"] == dataset[0]:
        y_pred, y_test, model = test_iris(model)
    elif CONFIG["dataset"] == dataset[1]:
        y_pred, y_test, model = test_give_me_credits(model)
    elif CONFIG["dataset"] == dataset[2]:
        y_pred, y_test, model = test_adult(model)
    elif CONFIG["dataset"] == dataset[3]:
        y_pred, y_test, model = test_default_credit_client(model)
    elif CONFIG["dataset"] == dataset[4]:
        y_pred, y_test, model = test_aug_data(model)
    elif CONFIG["dataset"] == dataset[5]:
        y_pred, y_test, model = test_texas(model) # TODO make
    elif CONFIG["dataset"] == dataset[6]:
        y_pred, y_test, model = test_purchase(model)
    if rank == PARTY_ID.SERVER:
        # model.log_info()
        import pickle
        pickle.dump({"model":model, "y_pred":y_pred, "y_test":y_test}, open( "debug.p", "wb"))
        # target_model = pickle.load(open(TARGET_MODEL_NAME, "rb"))

        acc, auc = model.evaluatePrediction(y_pred, y_test, treeid=99)    
        print("Prediction: ", acc, auc)

## Plotting and Evaluating